In [1]:
import pandas as pd
df=pd.read_csv("test.csv")
df.head()


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [114]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [115]:
null_columns = df.isnull().sum()
print(f'Total number of columns with null values: {null_columns[null_columns > 0].count()}')
print(null_columns[null_columns > 0])

Total number of columns with null values: 12
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64


In [116]:
for column in null_columns[null_columns > 0].index:
    if df[column].dtype == 'object':
        mode_value = df[column].mode()[0]
        df[column] = df[column].fillna(mode_value)
        df[column] = df[column].astype(str)  

C:\Users\M.Junaid\AppData\Local\Temp\ipykernel_3932\1195301289.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[column] = df[column].fillna(mode_value)  # Fill missing values
C:\Users\M.Junaid\AppData\Local\Temp\ipykernel_3932\1195301289.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[column] = df[column].fillna(mode_value)  # Fill missing values


In [117]:
null_columns = df.isnull().sum()
print(f'Total number of columns with null values: {null_columns[null_columns > 0].count()}')
print(null_columns[null_columns > 0])

Total number of columns with null values: 6
Age             float64
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
dtype: object


In [118]:
for column in null_columns[null_columns > 0].index:
    if df[column].dtype in ['int64', 'float64']:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        if df[(df[column] < lower_bound) | (df[column] > upper_bound)].empty:

            mean_value = df[column].mean()
            df[column].fillna(mean_value,inplace=True)
        else:
            
            values = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)][column]
            df[column].fillna(values.mean(),inplace=True)
            df.loc[(df[column] < lower_bound) | (df[column] > upper_bound), column] = values.mean()







C:\Users\M.Junaid\AppData\Local\Temp\ipykernel_3932\2959870808.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(values.mean(),inplace=True)
C:\Users\M.Junaid\AppData\Local\Temp\ipykernel_3932\2959870808.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For 

In [119]:
null_columns = df.isnull().sum()
print(f'Total number of columns with null values: {null_columns[null_columns > 0].count()}')
print(null_columns[null_columns > 0])

Total number of columns with null values: 0
Series([], dtype: int64)


In [120]:
from sklearn.preprocessing import LabelEncoder
object_columns = df.select_dtypes(include='object').columns.tolist()
object_columns.remove('PassengerId')

label_encoder = LabelEncoder()
for column in object_columns:
    df[column] = label_encoder.fit_transform(df[column].astype(str))

df.dtypes






PassengerId      object
HomePlanet        int64
CryoSleep         int64
Cabin             int64
Destination       int64
Age             float64
VIP               int64
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name              int64
dtype: object

In [121]:
from sklearn.preprocessing import StandardScaler
X=df.drop(['PassengerId','CryoSleep','VIP'], axis=1)
scaler = StandardScaler()
df[X.columns]=scaler.fit_transform(df[X.columns])

test_data=df.drop('PassengerId', axis=1)





In [122]:
import joblib

model = joblib.load('Mymodel.pkl')




In [123]:
predictions = model.predict(test_data)
submission_df = pd.DataFrame({'PassengerId': df['PassengerId'], 'Transported': predictions})

submission_df.to_csv('submission_file.csv', index=False)
